In [ ]:
# Install the pycoco library
!pip install pycocotools

In [ ]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import os
import random
import cv2
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Define the image and annotation paths
train_imgs_path = '../input/food-recognition-challenge/train-v0.4/train/images'
train_anns_path = '../input/food-recognition-challenge/train-v0.4/train/annotations.json'

test_imgs_path = '../input/food-recognition-challenge/test_images-v0.4/val/images'
test_anns_path = '../input/food-recognition-challenge/test_images-v0.4/val/annotations.json'

# Reference to Image Segmentation

https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html

In [ ]:
# Function to load JSON into a COCO api
def getCOCO(anns_path):
    # Initialize the COCO api for instance annotations
    coco = COCO(anns_path)
    
    # Load the categories in a variable
    catIDs = coco.getCatIds()
    cats = coco.loadCats(catIDs)
    
    # Print number of categories
    nms = [cat['name'] for cat in cats]
    
    # Create a dataframe of the count of each category
    df = pd.DataFrame(columns=['Category', 'ID', 'Count'])
    
    # Add each category and its count row by row
    for i, catID in enumerate(catIDs):
        imgIds = coco.getImgIds(catIds=catID)
        df.loc[i] = [nms[i]] + [catID] + [len(imgIds)]
    
    return coco, df

In [ ]:
# Load the annotations of the image dataset
train_coco, train_classes = getCOCO(train_anns_path)
test_coco, test_classes = getCOCO(test_anns_path)

In [ ]:
def getMask(image_id, coco, classes, image_size):
    # Create a zero array with the given size and number of classes
    mask = np.zeros((image_size[0], image_size[1], 274))

    annIds = coco.getAnnIds(int(image_id))
    anns = coco.loadAnns(annIds)

    for i, ann in enumerate(anns):
        # Get the binary mask for the annotation
        binary = cv2.resize(coco.annToMask(ann), image_size)

        # Get the channel index for the annotation
        channel = classes[classes.ID == ann['category_id']].index[0] + 1

        # Update the channel of the annotation
        mask[:, :, channel] = binary

        # Update the background channel of the annotation
        if i == 0:
            mask[:, :, 0] = np.logical_or(mask[:, :, 0], np.logical_not(binary))
        else:
            mask[:, :, 0] = np.logical_and(mask[:, :, 0], np.logical_not(binary))

    return mask

In [ ]:
def getGenerator(img_folder, coco, classes, image_size, batch_size):
    c = 0
    n = os.listdir(img_folder)
    random.shuffle(n)
    
    while (True):
        img_batch = np.zeros((batch_size, image_size[0], image_size[1], 3)).astype('int')
        mask_batch = np.zeros((batch_size, image_size[0], image_size[1], 274)).astype('float')

        for i in range(c, c + batch_size):
            img = cv2.imread(img_folder + '/' + n[i])
            img =  cv2.resize(img, image_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_batch[i-c] = img

            mask = getMask(n[i][1:-4], coco, classes, image_size)
            mask_batch[i-c] = mask

        c += batch_size
        if(c + batch_size >= len(os.listdir(img_folder))):
            c = 0
            random.shuffle(n)

        yield img_batch, mask_batch

In [ ]:
# Get the generators from the paths
IMAGE_SIZE = (32, 32)
BATCH_SIZE = 300

train_gen = getGenerator(train_imgs_path, train_coco, train_classes, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
train_length = len(os.listdir(train_imgs_path))
print('Number of training images: {}'.format(train_length))

test_gen = getGenerator(test_imgs_path, test_coco, test_classes, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
test_length = len(os.listdir(test_imgs_path))
print('Number of test images: {}'.format(test_length))

In [ ]:
# Install and import the segmentation models library
!pip install segmentation_models
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm

In [ ]:
# Define the model to be trained
model = sm.Unet('resnet34', encoder_freeze=True, classes=274, activation='softmax')
model.compile('Adam', loss=sm.losses.cce_dice_loss, 
              metrics=[sm.metrics.iou_score, sm.metrics.precision, sm.metrics.recall])

In [ ]:
# # Load the trained model weights
# weights_path = '../input/food-recognition-model/weights.h5'
# model.load_weights(weights_path)

In [ ]:
# Train the defined model on the dataset
history = model.fit(train_gen, steps_per_epoch=train_length//BATCH_SIZE, epochs=10)

In [ ]:
# Save the model to the output
model.save_weights('weights.h5')

In [ ]:
# Evaluate the model on the test generator
scores = model.evaluate(test_gen, steps=test_length//BATCH_SIZE)
print('\nLoss: {}'.format(round(scores[0], 3)))
print('Average IoU: {}'.format(round(scores[1], 3)))
print('Average Precision: {}'.format(round(scores[2], 3)))
print('Average Recall: {}'.format(round(scores[3], 3)))